# Introduction to functional programming in Python
<table style="margin-top:30px; border:none" align="left">
    <tr style="border:none"><td style="border:none"><p>Francesco Bruni</p></td></tr>
    <tr style="border:none" style="width:100%">
        <td style="border:none" style="width:90%"><p>@brunifrancesco</p></td>
    </tr>
</table>


## Who am I

- MSc Student in Telecommunication Engineering @ Poliba
- Despite a Java background, I prefer Python whenever possible
- I'm not a computer scientist

## Agenda

- Why functional programming
    - Everything's object
    - Laziness: why evaluation matters
    - Immutable data
    - Recursion and/or cycles
    - Pattern matching
- Mixing OOP with FP
- FP suitable patterns
- Conclusions


## Disclaimer

<p style="text-align: center; font-size: 40px"><strong>I'm not an hoover seller</strong></p>


# Why functional programming

- Think in terms of **functions**
- Enfatizes using *function evaluation* instead of *state evolution*
- Testing is (more) easy
- A new viewpoint is required

## (MATH) What is a function?

A **relation** from a set of inputs (X) to a set of possible outputs (Y) where **each** input is related to **exactly** one output.

<center>
<p style="text-align:center; font-size:30px;">f: X → Y</p>
<img src="images/function.svg" />
</center>

In [1]:
import random
def sum_imperative():
    res = 0
    for n in [random.random() for _ in range(100)]:
        res += n
    return res

def sum_functional():
    return sum(random.random() for _ in range(100))


# Functional features in Python

- Not all functional patterns apply to Python
- Hybrid approach is often requested
- Other libraries needs to be (eventually) integrated



## First class (or high order) functions 

- Since everything's object
- Functions are objects too (with fields and methods)


## High order functions

- Functions accepting functions as params
- Functions returning other functions
- **filter**, **map**, **reduce** (now in *itertools* module)

## (MATH) Function composition in math

A pointwise application of one function to the result of another to produce a third function. 

Given:
- f : X → Y 
- g : Y → Z 
          
<p style="text-align:center; font-size:40px">$g\circ f$ : X → Z</p>


In [12]:

def mapper_func(lst, reduce_func):
    """
    Apply a function to each member of <lst>
    """
    return map(reduce_func, lst)

for _ in range(0, 100):
    assert list(mapper_func([1,2,3], str))  == ["1", "2", "3"]
    assert list(mapper_func(["1","2","3"], int))  == [1, 2, 3]

In [16]:
def check_if_neg(value):
        """
        Check if values is neg
        """
        return value > 0
    
def filter_negative_numbers(lst):
    """
    Filter negative numbers from <lst>
    """
    return filter(check_if_neg, lst)

assert list(filter_negative_numbers([-3, 4, 5, -10, 20])) == [4, 5, 20]

## Pure functions

- Functions cannot include any assignement statement
- **lambda functions** are pure functions
- No side effects
- What about default param values? 

## (MATH) λ calculus

- A formal system to fomally analyze functions and their calculus.
- It deals with rewriting functions with simplified terms
- A formal definition of λ function:

<center style="font-size:20px">Λ ::= X |(ΛΛ)|λX.Λ</center>

http://infoteorica.weebly.com/uploads/1/7/8/9/17895653/lambda_rid.pdf

In [8]:
import random
def filter_out(result=[random.random() for _ in range(0, 5)]):
    
    exclude = map(lambda item: item ** 2, range(30))

    result = filter(lambda item: item not in exclude, result)
    
    sorted_result = sorted(result, key=lambda item: str(item)[1])
    
    return map(lambda item: round(item, 2), sorted_result)
    
cast_to_float = lambda number: float(number) # discouraged by PEP8
filter_out()

## Practical considerations of λ functions

- Inline functions
- Concise
- No need of defining a *one time* functions
- Overusing them is not a solution
- assigning λ function to a variable is discouraged (PEP8)

## Class for function composition
Handle class creation via a new class syntax, allowing a "static" configuration


In [28]:
from collections.abc import Callable
from random import random
from statistics import mean

class MapReduceFunction(Callable):
    """
    'Chain' two functions to perform map reduce;
    the class returns a new callable object
    """
    def __init__(self, map_function, reduce_function):
        self.map_function = map_function
        self.reduce_function = reduce_function
    
    def __call__(self, value):
        return map(lambda item: self.reduce_function(item), self.map_function(value))

data = [round(random()*10, 3) for _ in range(0, 23)]
mr = MapReduceFunction(
    map_function=lambda item: zip(*[iter(data)] * 7), 
    reduce_function=lambda item: max(item)
)
list(mr(data))

[9.593, 8.297, 6.521]

# Immutable vs mutable data structure

- Variables don't **vary** any more
- Will your program still working?

In [91]:
value = 100

def change_f_value(new_f_value=5):
    value = new_f_value
    print("Value in function %s" %value)
    
print("Initialized value %s "%value)
change_f_value()
print("Final value %s "%value)

Initialized value 100 
Value in function 5
Final value 100 


### Function scopes

- Values inside functions exist in the *context* of functions
  - Check this evaluating their own *hash value*
- What if we wanted change the *value* variable?

In [7]:
class Foo:
    def __init__(self, value):
        self.value = value
        
foo_obj = Foo(value=10)

def func(obj):
    obj.value = 3

print("Object ID: %i" %id(foo_obj))
print("Object 'value' field before applying function: %i" %foo_obj.value)
func(foo_obj)
print("Object 'value' field after applying function: %i" %foo_obj.value)
print("Object ID: %i" %id(foo_obj))

Object ID: 140576565540736
Object 'value' field before applying function: 10
Object 'value' field after applying function: 3
Object ID: 140576565540736


### Data mutation

- *foo_obj* didn't change
- **foo_obj.value** changed!
- So, foo_obj changed or not? If so, can you always determine who changed it?

### Closures

## Immutability

- Let's create a new object, instead of modifying an existing one

In [ ]:
import random
import pprint
from collections import namedtuple

MyObj = namedtuple("MyClassReplacement",("some_string", "my_smart_function"))
o = MyObj(some_string=str(random.random() + 4), my_smart_function=lambda item: float(item)*3)
assert(o.my_smart_function(o.some_string) == float(o.some_string) * 3)

# guess this
#o.some_string = "aaa"



## Strict vs not strict evaluation

- Strict evaluation requires that all operators needs to be evaluated
- Non strict (or lazy) evaluation, evaluates expression if and when requested

## (MATH) Truth table


<table>
<tr>
<td style="width:50px; text-align:center"><strong>p</strong></td>
<td style="width:50px; text-align:center"><strong>q</strong></td>
<td style="width:50px; text-align:center"><strong>p & q</strong></td>
</tr>
<tr>
<td style="text-align:center">T</td>
<td style="text-align:center">T</td>
<td style="text-align:center">T</td>
</tr>
<tr>
<td style="text-align:center">T</td>
<td style="text-align:center">F</td>
<td style="text-align:center">F</td>
</tr>
<tr>
<td style="text-align:center">F</td>
<td style="text-align:center">T</td>
<td style="text-align:center">F</td>
</tr>
<tr>
<td style="text-align:center">F</td>
<td style="text-align:center">F</td>
<td style="text-align:center">F</td>
</tr>
</table>


In [68]:
import random

generate_random_list = lambda size: [random.choice([True, False]) for _ in range(0, size)]

def all_true_values(lst):
    print("evaluating ALL true values")
    return all(lst)

def any_true_value(lst):
    print("evaluating ANY true values")
    return any(lst)

all_true_values(generate_random_list(size=10)) and any_true_value(generate_random_list(size=10))
print("+++++++++++")
all_true_values(generate_random_list(size=10)) or any_true_value(generate_random_list(size=10))




evaluating ALL true values
+++++++++++
evaluating ALL true values
evaluating ANY true values


True

## Use case: Python iterables structures

- Creating lists requires time/space
- What if you don't need it anymore?
- Be lazy: use functions to **generate** the element you need, also in iterables

In [94]:
import random

def _lc(size=100):
    return [random.random() for _ in range(0, 5)]

def _iter(size=100):
    return iter([random.random() for _ in range(0, 100)])

def lazy_1(size=100):
     for item in range(10, size):
        if not item % 2 and not str(item).endswith("4"):
            yield item

def lazy_2(size=100):
    yield from (r for r in range(10, size) if not r % 2 and not str(r).endswith("4"))

print(_lc())
print(_iter())
print(lazy_1())
print(lazy_2())

[0.4614800399496741, 0.6936061338751625, 0.6455814839628797, 0.6036356634912029, 0.5594124924886037]
<generator object lazy_1 at 0x11092f3a8>
<generator object lazy_2 at 0x11092f3a8>


## Recursion vs loop
- Functional programming requires a recursion approach vs loop iteration
- Python suffers by recursion limit
- Python does not offer any tail call optimization

In [34]:
def facti(n):
    if n == 0: return 1
    f= 1
    for i in range(2,n):
        f *= i
    return f

def fact_nt(n):
    if n == 0: return 1
    else: return n*fact(n-1)

def fact(n, acc=1):
    if n == 0:
        return acc
    return fact(n-1, acc*n)

%time facti(100)
%time fact_nt(100)
%time fact(100)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 33.9 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 97.8 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 66.5 µs


93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

## Currying

Multiple arguments functions mapped to single arguments functions

In [19]:
from random import randrange

def mult(a, b, c ):
    def wrapper_1(b):
        def wrapper_2(c):
            return a*b*c
        return wrapper_2(c)
    return wrapper_1(b)


def mult_2(a):
    def wrapper_1(b):
        def wrapper_2(c):
            return a*b*c
        return wrapper_2
    return wrapper_1


assert(mult(2,4,8) == 64)
assert(mult_2(2)(3)(4) == 24)

## Partials
Python provides "partial" functions for manual currying

In [1]:
from functools import reduce
import operator
import random

def two_random_sum():
    return reduce(operator.sum, [random.random()]*2)

def three_random_product():
    return reduce(operator.mul, [random.random()]*3)

def four_random_sub():
    return reduce(operator.sub, [random.random()]*4)

def five_random_pow ():
    return reduce(operator.pow, [random.random()]*5)


In [24]:
def handle_random_numbers(size, function):
    return reduce(function, [random.random()]*size)

two_random_sum = partial(handle_random_numbers, size=2)
three_random_sum = partial(handle_random_numbers, size=3)

two_random_pow = partial(handle_random_numbers, size=2, function=operator.pow)
five_random_product = partial(handle_random_numbers, size=5, function=operator.mul)


## Decorator Pattern

- Return a modified version of a decorated function
- Add properties at runtime, before using the actual decorated function

In [30]:
from functools import wraps
from functools import partial

def get_ned_data(n):
    def get_doubled_data(func, *args, **kwargs):
        @wraps(func)
        def _inner(*args, **kwargs):
            kwargs["multiplied_by_n_param"] = kwargs["initial_param"]*n
            return func(*args, **kwargs)
        return _inner
    return get_doubled_data

In [42]:
@get_ned_data(n=2)
def double_func(*args, **kwargs):
    assert(kwargs["multiplied_by_n_param"] == kwargs["initial_param"]*2)

@get_ned_data(n=3)
def triple_func(*args, **kwargs):
    assert(kwargs["multiplied_by_n_param"] == kwargs["initial_param"]*3)
    
double_func(initial_param=3)
triple_func(initial_param=5)

# FP patterns

- Option pattern
- Memoization
- Actor model

## Option pattern

- How to handle error in functional programming?
- How to implement pipelined ops?

In [72]:
class Container:
    def __init__(self, value=None):
        self.value = value
    
    def apply(self, function):
        try:
            return Full(function(self.value))
        except Exception as e:
            return Empty()
    

class Empty(Container):
    def apply(self, value):
        return Empty()

    def __str__(self):
        return "Container's empty"

class Full(Container):
    def __str__(self):
        return self.value
    
    def get_or(self, none_value=None):
        return self.value or none_value

In [84]:
map_function=lambda item: zip(*[iter(data)] * 7)
reduce_function=lambda item: max(item)

print(Container("some_string")
      .apply(lambda item: len(item))\
      .apply(lambda item: str(item))
      )
result = Container(round(random()*10, 3) for _ in range(0, 23))\
    .apply(map_function)\
    .apply(reduce_function)\
    .get_or(0)
print(result)

11
(8.297, 7.963, 6.004, 5.352, 3.367, 3.062, 7.801)


In [81]:
from fn.monad import optionable
from collections import namedtuple

def get(request, *args, **kwargs):
    @optionable
    def _get_values(data):
        return data.get("values", None)
    _split = lambda item: item.split(",")
    _strip = lambda item: item.replace(" ", "")
    _filter = lambda item: list(filter(lambda i: i, item))
    values = _get_values(request.body).map(_strip).map(_split).map(_filter).get_or(["v1,v2"])
    return values

req_class = namedtuple("Request", ("body",))

request_1 = req_class(dict(values="v1, v2,v3"))
request_2 = req_class(dict(values="v1,v2,v3 "))
request_3 = req_class(dict(values="v1, v2,v3, "))

assert(get(request_1) == ['v1', 'v2', 'v3'])
assert(get(request_2) == ['v1', 'v2', 'v3'])
assert(get(request_3) == ['v1', 'v2', 'v3'])

## Memoization

## Actors

- Message passing pattern
- Actors receive a message, do something, return new messages (or None)
- They behave like humans

# Useful libraries

- **fn.py**
- **pyMonads**
- **pyPrsisten**
- **toolz**
- **pykka**
- **pulsar**
- **cleveland**
- **underscore.py** (ported from underscore_js)

# Summary
- Functional programming for writing more succint code
- Change viewpoint: think in terms of function (and start applying maths to code)
- Python seems for dummies until you show some cool features

# Questions?